<a href="https://colab.research.google.com/github/cristianbossolasco/Inflacion-Dolar-Argentina/blob/main/Dolar_vs_IPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
# Inflacion prevista para el proximo mes
def get_inflacion_prevista(porcentaje_provisto, df):
  max_index = df.idxmax()
  valor = df.loc[max_index].values[0][0]

  index_new = max_index[0] + relativedelta(months=1)
  valor_new = round(valor + valor * porcentaje_provisto, 4)

  return index_new, valor_new

def get_data():
  url = 'https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_03_24.xls'
  excel_file = pd.ExcelFile(url)
  sheet_name = 'Índices IPC Cobertura Nacional'
  skip_rows = 5  # Omitir las primeras 5 filas
  num_rows = 26  # Leer las siguientes 26 filas
  data = excel_file.parse(sheet_name, skiprows=skip_rows, nrows=num_rows)

  filtro = data['Total nacional'] == 'Nivel general'
  ipc_general = data[filtro].set_index('Total nacional').T


  # Descomentar las dos lineas de abajo si se conoce la inflacion prevista del mes en curso
  # index_new, valor_new = get_inflacion_prevista(0.10, ipc_general)
  # ipc_general.loc[index_new] = valor_new

  # calculo los % de variacion mensual, trimestral y interanual
  #ipc_general['Variación mensual'] = round(ipc_general['Nivel general'].pct_change() * 100, 1)
  #ipc_general['Variación Timestral'] = round(ipc_general['Nivel general'].pct_change(periods=3) * 100, 1)
  #ipc_general['Variación Anual'] = round(ipc_general['Nivel general'].pct_change(periods=12) * 100, 1)
  for periodos in range(1,13):
    ipc_general[f'Inf_{periodos}'] = round(ipc_general['Nivel general'].pct_change(periods=periodos) * 100, 1)

  # Renombro las columnas
  ipc_general = ipc_general.rename(columns={'Nivel general': 'Valor'})

  return ipc_general

def get_historico_dolar(tipo, fecha_inicial, fecha_final):
  response = requests.get(f'https://mercados.ambito.com//dolar/{tipo}/grafico/{fecha_inicial}/{fecha_final}')

  data  = response.content

  # Decodificar la cadena de bytes y convertirla en una lista
  decoded_data = data.decode('utf-8')
  data_list = json.loads(decoded_data)

  # Crear el dataframe
  df = pd.DataFrame(data_list[1:], columns=data_list[0])

  # Me aseguro que la fecha tenga formato fecha
  df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y')

  return df

def get_cotizaciones_dolar(fecha_desde, fecha_hasta):
  df_dolar_oficial = get_historico_dolar('formal', fecha_desde, fecha_hasta)
  df_dolar_blue = get_historico_dolar('informal', fecha_desde, fecha_hasta)

  # elimino duplicados
  df_dolar_oficial_sin_dupl = pd.DataFrame(df_dolar_oficial.groupby('fecha')['Dólar Oficial'].max()).sort_index()
  df_dolar_blue_sin_dupl = pd.DataFrame(df_dolar_blue.groupby('fecha')['Dólar Informal'].max()).sort_index()

  # Utiliza el método resample para obtener el último valor de cada mes
  df_dolar_oficial_mes = df_dolar_oficial_sin_dupl.resample('M').last()
  df_dolar_blue_mes = df_dolar_blue_sin_dupl.resample('M').last()

  # unifico los dataframe
  df_dolar = df_dolar_oficial_mes.merge(
                df_dolar_blue_mes,
                left_on='fecha',
                right_on='fecha'
            )
  df_dolar.rename(columns={'Dólar Oficial': 'Oficial', 'Dólar Informal' : 'Blue'}, inplace=True)

  # Calcula el porcentaje de variación respecto al mes anterior
  df_dolar['Variacion_Oficial'] = df_dolar['Oficial'].pct_change() * 100
  df_dolar['Variacion_Blue'] = df_dolar['Blue'].pct_change() * 100
  df_dolar = df_dolar.reset_index()

  return df_dolar

In [3]:
fecha_desde, fecha_hasta = '2000-01-01', '2024-3-31'

df_dolar = get_cotizaciones_dolar(fecha_desde, fecha_hasta)
df_dolar_2 = df_dolar[['fecha', 'Blue', 'Variacion_Blue']].copy()
df_dolar_2['fecha'] = df_dolar_2['fecha'].dt.to_period('M')

ipc = get_data().reset_index().rename(columns={'index': 'Periodo'})
ipc_2 = ipc[['Periodo',	'Valor',	'Inf_1']].copy()
ipc_2['Periodo'] = ipc_2['Periodo'].dt.to_period('M')

merged_data = pd.merge(ipc_2, df_dolar_2, left_on='Periodo', right_on='fecha', how='inner')
merged_data = merged_data[['Periodo', 'Valor', 'Blue', 'Inf_1', 'Variacion_Blue']]
merged_data = merged_data.sort_index()
merged_data = merged_data.sort_values(by='Periodo', ascending=False)
merged_data = merged_data.rename(columns={'Periodo': 'Periodo', 'Valor': 'Indice', 'Blue' : 'Dolar', 'Inf_1' : 'Inflacion', 'Variacion_Blue': 'Variacion_Dolar'})
merged_data.head(12)

,Periodo,Indice,Dolar,Inflacion,Variacion_Dolar
86,2024-02,4825.7881,1045.0,13.2,-12.552301
85,2024-01,4261.5324,1195.0,20.6,16.585366
84,2023-12,3533.1922,1025.0,25.5,13.259669
83,2023-11,2816.0628,905.0,12.8,-1.630435
82,2023-10,2496.2730,920.0,8.3,15.000000
81,2023-09,2304.9242,800.0,12.7,8.843537
80,2023-08,2044.2832,735.0,12.4,33.636364
79,2023-07,1818.0838,550.0,6.3,11.336032
78,2023-06,1709.6115,494.0,6.0,0.202840
77,2023-05,1613.5895,493.0,7.8,5.567452
